In [126]:
import time
import numpy as np
import tracemalloc
from scipy.io import wavfile

In [2]:
def pad_zeros_to(input, new_length):
    output = np.zeros((new_length,))
    output[:input.shape[0]] = input
    return output

In [3]:
def fft_convolution(input, filter, K=None):
    in_size = input.shape[0]
    filt_size = filter.shape[0]
    out_size = filt_size + in_size - 1

    if K is None:
        K = 1 << (int(np.log2(Ny-1)) + 1)

    X = np.fft.fft(pad_zeros_to(input, K))
    H = np.fft.fft(pad_zeros_to(filter, K))

    Y = np.multiply(X, H)

    out = np.real(np.fft.ifft(Y))

    return out[:out_size]

In [122]:
def load_wav(file):
    samplerate, data = wavfile.read(file)
    data = data.astype(np.float32)
    data = data/max(abs(data))
    return data

In [128]:
def overlap_save(input, filter, block_length, K=None):
    '''
    This partitions both the filter and the input and performs a convolution.
    block_length should be a power of 2, and K should be twice the block length
    '''
    # TODO update so the arrays are 32 bit floats
    # pad input it can be split uniformly into blocks
    num_input_blocks = np.ceil(len(input)/block_length).astype(int)
    padded_input = pad_zeros_to(input, num_input_blocks * block_length)

    # pad filter
    num_filt_blocks = np.ceil(len(filter)/block_length).astype(int)
    padded_filter = pad_zeros_to(filter, num_filt_blocks * block_length)

    # doing the convolution
    input_buffer = np.zeros((block_length*2,))
    buffer = np.zeros((block_length*2,))
    output_buffer = np.zeros((block_length*2,))
    output = []

    # zeropad filters and perform fft on them
    sub_filters = np.zeros((num_filt_blocks, block_length*2))
    i = 0
    for f in sub_filters:
        f[:block_length] = padded_filter[i:i+block_length]
        i += block_length
        f = np.fft.fft(f)

    i = 0
    while i < block_length * num_input_blocks:
        # put B into buffer
        input_buffer[:block_length] = input_buffer[block_length:]
        input_buffer[block_length:] = padded_input[i:i+block_length]
        # perform fft on B
        buffer = np.fft.fft(input_buffer)
        for f in sub_filters:
            result = np.multiply(buffer,f)
            output_buffer = np.add(output_buffer, np.multiply(buffer,f))
        y = np.real(np.fft.ifft(output_buffer))
        output = np.concatenate((output, y[block_length:])).astype(np.float32)
        i+=block_length

    return output

In [106]:
def create_reverb_ir(rvb_size = int(0.1*44100)):  # reverb lasts for 0.006 seconds
    reverb_ir = np.zeros(rvb_size, dtype='float32')  
    reverb_ir[0] = 1.0                     # add 4 reflections to the wave
    reverb_ir[int(rvb_size / 2)] = 0.8   
    reverb_ir[int(rvb_size / 4)] = 0.6
    reverb_ir[int(rvb_size * 3 / 4)] = 0.6
    return reverb_ir

In [130]:
data = load_wav("audio.wav")

reverb = create_reverb_ir()

output = overlap_save(data, reverb, 1024, 2048)

output = output/max(abs(output))

wavfile.write("reverb_audio.wav", 44100, output)



